# Lab 10.3.2 MNIST-CNN with Visdom

**Jonathan Choi 2021**

**[Deep Learning By Torch] End to End study scripts of Deep Learning by implementing code practice with Pytorch.**

If you have an any issue, please PR below.

[[Deep Learning By Torch] - Github @JonyChoi](https://github.com/jonychoi/Deep-Learning-By-Torch)

Here we will create the CNN model to classify the MNIST with visualizing tool of Visdom.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.init as init
import random

## Import Visdom

In [3]:
import visdom

vis = visdom.Visdom()
vis.close(env = "main")

Setting up a new session...


''

## Define loss_tracker

In [4]:
def loss_tracker(loss_plot, loss_value, num):
    #num, loss_value are Tensor
    vis.line(X = num, Y=loss_value, win=loss_plot, update='append')

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed_all(1)

## Set Hyperparameters

In [6]:
learning_rate = 0.001
training_epochs = 15
batch_size = 32

## Load MNIST Data

In [7]:
mnist_train = datasets.MNIST(download = True,
                             train = True,
                             transform = transforms.ToTensor(),
                             root = 'MNIST_data/')
mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            train = False,
                            download = True,
                            transform = transforms.ToTensor())

C:\Users\buddhalight\envs\buddhalight\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, shuffle = True, drop_last = True, batch_size = batch_size)

In [9]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        #print(out.shape) torch.Size([1, 128, 3, 3])
        out = out.view(out.size(0), -1)
        #print(out.shape) torch.Size([1, 1152])
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [10]:
model = CNN().to(device)

value = torch.Tensor(1,1,28,28).to(device)
print(model(value).shape)

C:\Users\buddhalight\envs\buddhalight\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([1, 10])


In [11]:
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Make Plot

In [12]:
loss_plt = vis.line(Y = torch.Tensor(1).zero_(), opts = dict(title = 'loss_tracker', legend=['loss'], showlegend = True))

## Train with loss_tracker

In [13]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        #prediction
        pred = model(X)

        #cost
        cost = F.cross_entropy(pred, Y)

        #Reduce the cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost

    avg_cost = avg_cost / total_batch

    print('Epoch: {} / {}, Cost: {:.6f}'.format(epoch+1, training_epochs, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished')

Epoch: 1 / 15, Cost: 0.116303
Epoch: 2 / 15, Cost: 0.040117
Epoch: 3 / 15, Cost: 0.028779
Epoch: 4 / 15, Cost: 0.021760
Epoch: 5 / 15, Cost: 0.017429
Epoch: 6 / 15, Cost: 0.014483
Epoch: 7 / 15, Cost: 0.012259
Epoch: 8 / 15, Cost: 0.010881
Epoch: 9 / 15, Cost: 0.010224
Epoch: 10 / 15, Cost: 0.007701
Epoch: 11 / 15, Cost: 0.009009
Epoch: 12 / 15, Cost: 0.006754
Epoch: 13 / 15, Cost: 0.007524
Epoch: 14 / 15, Cost: 0.007479
Epoch: 15 / 15, Cost: 0.005707
Learning Finished


![](./imgs/plot1.png)

In [19]:
with torch.no_grad():
    print(mnist_test.data.shape)
    print(mnist_test.data[0].dtype)
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)#if not .float() error : RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same
    print(X_test.shape)
    print(X_test[0].dtype)
    Y_test = mnist_test.targets.to(device)

    pred = model(X_test).to(device)
    correct_prediction = (torch.argmax(pred, 1) == Y_test)
    accuracy = correct_prediction.float().mean()

    r = random.randint(0, len(X_test.data) - 1)
    X_single_prediction = X_test[r:r+1]
    Y_single_prediction = Y_test[r:r+1]
    print(X_single_prediction.shape) # if [r] => torch.size([1, 28, 28]) # if [r:r+1] => torch.size([1,1,28,28])
    
    print('Accuracy: {:.9f}'.format(accuracy.item()))
    print('Prediction: {}, Label: {}'.format(torch.argmax(model(X_single_prediction)), Y_single_prediction.item()))

    

torch.Size([10000, 28, 28])
torch.uint8
torch.Size([10000, 1, 28, 28])
torch.float32
torch.Size([1, 1, 28, 28])
Accuracy: 0.949599981
Prediction: 0, Label: 0
